# CoSim - RTED

In [1]:
import numpy as np

import andes
import ams

import pandas as pd

import json

import cvxpy as cp

In [2]:
ams.__version__

'0.6.4.post83.dev0+g065281c'

In [3]:
andes.config_logger(stream_level=20)

In [4]:
sa = andes.load(andes.get_case('kundur/kundur_coi.xlsx'),
                setup=True,
                no_output=True,
                )


Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Parsing input file "/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/andes/cases/kundur/kundur_coi.xlsx"...
Input file parsed in 0.1082 seconds.
System internal structure set up in 0.0220 seconds.


In [5]:
sa.PFlow.run()

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0041 seconds.
0: |F(x)| = 14.9282832
1: |F(x)| = 3.608627841
2: |F(x)| = 0.1701107882
3: |F(x)| = 0.002038626956
4: |F(x)| = 3.745104035e-07
Converged in 5 iterations in 0.0052 seconds.


True

In [6]:
_ = sa.TDS.init()

Initialization for dynamics completed in 0.0209 seconds.
Initialization was successful.


In [7]:
sa.COI.Mw

ConstService: COI.Mw, v=[0.5 0.5 0.5 0.5]

In [8]:
sa.COI.wgen

ExtState: COI.wgen, a=[4 5 6 7], v=[1. 1. 1. 1.], e=[]

In [9]:
ams.config_logger(stream_level=10)

In [10]:
sp = ams.load(ams.get_case('ieee14/ieee14_rted.xlsx'),
              setup=True,
              )

Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
Input format guessed as xlsx.
Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/ieee14/ieee14_rted.xlsx"...
Input file parsed in 0.0446 seconds.
Adjust bus index to start from 0.
System set up in 0.0041 seconds.


In [11]:
sp.RTED.setup()

RTED data check passed.
- Generating symbols for RTED
Set constrs pb: self.constrs["pb"]=cp.sum(self.routine.pd.v) - cp.sum(self.pg) == 0
Set constrs lub: self.constrs["lub"]=self.routine.PTDF1.v @ (self.pg - self.routine.pd1.v) - self.routine.PTDF2.v @ self.routine.pd2.v - self.routine.rate_a.v <= 0
Set constrs llb: self.constrs["llb"]=- self.routine.PTDF1.v @ (self.pg - self.routine.pd1.v) + self.routine.PTDF2.v @ self.routine.pd2.v - self.routine.rate_a.v <= 0
Set constrs rbu: self.constrs["rbu"]=self.routine.prs.v @ self.pru - self.routine.du.v == 0
Set constrs rbd: self.constrs["rbd"]=self.routine.prs.v @ self.prd - self.routine.dd.v == 0
Set constrs rru: self.constrs["rru"]=self.pg + self.pru - self.routine.pmax.v <= 0
Set constrs rrd: self.constrs["rrd"]=-self.pg + self.prd - self.routine.pmin.v <= 0
Set constrs rgu: self.constrs["rgu"]=self.pg - self.routine.pg0.v - self.routine.R10.v <= 0
Set constrs rgd: self.constrs["rgd"]=-self.pg + self.routine.pg0.v - self.routine.R10.v <

True

In [12]:
sp.RTED.constrs

OrderedDict([('pb', pb: sum(pd) - sum(pg)),
             ('lub', lub: PTDF1 @ (pg - pd1) - PTDF2 * pd2 - rate_a),
             ('llb', llb: - PTDF1 @ (pg - pd1) + PTDF2 * pd2 - rate_a),
             ('rbu', rbu: prs @ pru - du),
             ('rbd', rbd: prs @ prd - dd),
             ('rru', rru: pg + pru - pmax),
             ('rrd', rrd: -pg + prd - pmin),
             ('rgu', rgu: pg - pg0 - R10),
             ('rgd', rgd: -pg + pg0 - R10)])

In [13]:
sp.RTED.run()

RTED completed in 0.0132 seconds with exit code 0.


In [14]:
sp.RTED.prd

Var: StaticGen.prd, a=[10 11 12 13 14], v=[0.01666667 0.01666667 0.05       0.         0.01666667]

In [ ]:
import cvxpy as cp

In [ ]:
help(sp.Variable)

In [ ]:
sp.RTED.prd

In [ ]:
sa = sp.to_andes(setup=False,
                 addfile=andes.get_case('ieee14/ieee14_wt3.xlsx'),
                 overwrite=True,
                 keep=False,
                 no_output=True,
                 )

sa.add("Toggle", dict(model='SynGen', dev="GENROU_4", t=11.0))

sa.setup()

In [ ]:
sp.ACOPF.run()

In [ ]:
sp.ACOPF.pg

In [ ]:
sp.dyn.send()

In [ ]:
sa.PFlow.run()

In [ ]:
idx_tg = sa.TGOV1.idx.v
vmax0 = sa.TGOV1.get(src='VMAX', attr='v', idx=idx_tg)
sa.TGOV1.set(src='VMAX', attr='v', idx=idx_tg, value=10 * vmax0)
vmin0 = sa.TGOV1.get(src='VMIN', attr='v', idx=idx_tg)
sa.TGOV1.set(src='VMIN', attr='v', idx=idx_tg, value=np.zeros_like(vmin0))

In [ ]:
sa.TDS.config.tf = 10
sa.TDS.run()

In [ ]:
sa.TDS.config.tf = 20
sa.TDS.run()

In [ ]:
sp.dyn.send()

In [ ]:
sp.dyn.receive()